## UCI SECOM Dataset
### Semiconductor manufacturing process dataset

##### Manufacturing process feature selection and categorization
What we will do in this notebook? 
#### In this Notebook we will do the training and testing pipeline. Later we will make a pipeline with .py depending on each step


data source: https://www.kaggle.com/datasets/paresh2047/uci-semcom?resource=download

1) Imports

In [1]:
# Libraries

import numpy as np
import pandas as pd

In [2]:
# Data 

df = pd.read_parquet('.\\data\\uci-secom.parquet')

In [3]:
df1

NameError: name 'df1' is not defined